In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/novozymes-enzyme-stability-prediction/sample_submission.csv
/kaggle/input/novozymes-enzyme-stability-prediction/wildtype_structure_prediction_af2.pdb
/kaggle/input/novozymes-enzyme-stability-prediction/train.csv
/kaggle/input/novozymes-enzyme-stability-prediction/test.csv
/kaggle/input/novozymes-enzyme-stability-prediction/train_updates_20220929.csv


## Physics behind Protein Stability

**DDG** : Delta Delta G (DDG) is a metric for predicting how a single point mutation will affect protein stability. The thermodynamic stability change upon mutation is measured by ΔΔG(T$_{r}$), i.e. the difference between the standard Gibbs folding free energies of the mutant (ΔG$^{mut}$) and wild-type (ΔG$^{wild}$) proteins at the reference temperature T$_{r}$:

$$  ΔΔG(T_{r}) = (ΔG^{mut})(T_{r}) - (ΔG^{wild})(T_{r}) $$

In general,

$$ G = Enthalpy \ \  (H) - Temperature \ \ (T) \ x \ Entropy \ \ (S)  $$

H is the internal energy of a protein. H decreases during protein folding because folding will cause packing of hydrophobics, optimized polar group orientation, and achieves a good proximity to ideal bond lengths and angles. S is the measure of order within a system. S of a protein becomes lower during protein folding because residue dynamics will be significantly decreased in comparison the unfolded state. However, decrease is in protein S will also cause an increase in S for solvent. The energy of an unfolded protein is nearly identical with a single point mutation. So, the
dominant factor in DDG is the energy difference of the folded state.


DDG results will fall into three categories:

1. **DDG > 0.5**: Positive results suggest that a mutation would be destabilizing. Most mutations will be positive or close to zero because proteins have evolved to be reasonably stable. These mutations are residues that you should usually avoid during design.

2. Things that are near 0 are within the noise range so should be considered neutral.

3. DDG < -0.5: Negative results suggest that the mutation would lead to a more stable protein. However, the environment at each position should be considered.

    a. If interacting molecules are not present in the model, such as at a known zinc binding site, then a seemingly favorable mutation will not be favorable in reality.
    
    b. A positions that has a lot of negative DDGs could mean that this position evolved a destabilizing residue because it is necessary for its catalytic activity, for binding another molecule, or because of another functionally relevant reason.
    
    c. Also, consider that this measures a single point mutation. Many times it requires multiple interacting mutations in order to achieve significant stability.

# ESM-2

ESM-2 is a transformer-based language model, and uses an attention mechanism to learn interaction patterns between pairs of amino acids in the input sequence.

The larger the value of the attention, the more impactful would be any mutation at that location. So the melting temperature is inversely proportional to the sum of the contact attentions.

In [33]:
# latest release
!pip install fair-esm

In [34]:
#set the edit_idx
import pandas as pd, numpy as np, nltk
import torch
import esm
wt = "VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK"
test_df = pd.read_csv("../input/novozymes-enzyme-stability-prediction/test.csv")
test_df.loc[1169, 'protein_sequence'] = wt[:-1]+"!" #1169 is the same as wt

test_df['edit_idx'] = test_df.apply(lambda x:[i for i in range(len(x['protein_sequence'])) if x['protein_sequence'][i] != wt[i]][0], axis = 1)



#From the quick start code here - https://github.com/facebookresearch/esm

model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t33_650M_UR50D")


# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results


# Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
data = [
    ("protein1", wt),
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=True)
token_representations = results["representations"][33]

# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
for i, (_, seq) in enumerate(data):
    sequence_representations.append(token_representations[i, 1 : len(seq) + 1].mean(0))

    
#all the code above is from ESM QuickStart
ac_sum = np.sum(np.array(results["contacts"][0]), axis=1) 
test_df['tm'] = test_df.apply(lambda x:1/(2*ac_sum[x['edit_idx']]), axis = 1)  
test_df[['seq_id', 'tm']].set_index("seq_id").to_csv("submission.csv")


Using cache found in /root/.cache/torch/hub/facebookresearch_esm_main
